In [1]:
DEARGADAOL_LANDING = "https://www.leighleat.com/sc%C3%A9alta/45"

In [2]:
import requests
from bs4 import BeautifulSoup

In [14]:
def get_page_list(url: str):
    PAGE_REQ = requests.get(url)
    if PAGE_REQ.status_code != 200:
        return None

    soup = BeautifulSoup(PAGE_REQ.content, "html.parser")

    pages = set()
    for row in soup.find_all("div", {'class': 'row'}):
        for anchor in row.find_all("a"):
            if anchor["href"].startswith("/pages/"):
                pages.add(f'https://www.leighleat.com{anchor["href"]}')
    return list(pages)

In [65]:
def get_page(url: str):
    PAGE_REQ = requests.get(url)
    if PAGE_REQ.status_code != 200:
        return None

    soup = BeautifulSoup(PAGE_REQ.content, "html.parser")

    audio_url = ""
    audio = soup.find("audio", {"id": "audio"})
    audio_source = audio.find("source")
    if audio_source:
        if audio_source["src"].startswith("http"):
            audio_url = audio_source["src"]
        elif audio_source["src"].startswith("/"):
            audio_url = f'https://www.leighleat.com{audio_source["src"]}'

    text_pieces = []
    content = soup.find("div", {"class": "story-page-content"})
    if not content:
        content = soup.find("div", {"class": "inner-container"})
        if content:
            for child in content.children:
                print("")
                for ent in child.__dict__:
                    if ent not in ["parent", "previous_element"]:
                        print(child.name, ent, child.__dict__[ent])
                if child.name == "h1" and "page-text" in child["class"]:
                    if child.text.strip() != "":
                        text_pieces.append(child.text.strip())
                elif child.name == "div" and child["class"] == "text":
                    print("div", child)
                    for subchild in child.children:
                        if subchild.name == "p":
                            print("para", subchild.text)
                            if subchild.text.strip() != "":
                                text_pieces.append(subchild.text.strip())
    if len(text_pieces) != 0:
        text = " ".join(text_pieces)
    else:
        text = content.text.strip()
    return audio_url, text
    

In [ ]:
get_page("https://www.leighleat.com/pages/475")

In [ ]:
get_page("https://www.leighleat.com/pages/2177")

In [26]:
test_audio = "https://www.leighleat.com/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaHBBclFuIiwiZXhwIjpudWxsLCJwdXIiOiJibG9iX2lkIn19--a6f03c6d6a35f802262cd6a4a26c3108ef6e8449/Cli%CC%81ona%20&%20an%20Clai%CC%81omh%20Cb%201%20lch%201.mp3"

In [27]:
aud = requests.get(test_audio)